In [ ]:
# 필요한 모듈 import
import pandas as pd
import numpy as np
import re
import math

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import spearmanr
from scipy.stats import pearsonr
from scipy.spatial import cKDTree
from sklearn.neighbors import BallTree
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 한글처리
plt.rcParams['font.family'] ='NanumGothic'
plt.rcParams['axes.unicode_minus'] =False

In [ ]:
# train/test data 불러오기
train_df = pd.read_csv('train_2016.csv', encoding='utf-8')
test_df = pd.read_csv('test_2016.csv', encoding='utf-8')

In [ ]:
# 거래가격분포 확인
fig = plt.figure(figsize=(10, 5))
sns.histplot(train_df['target'])
plt.title("거래가격 분포", fontsize=14)
ax = plt.gca()
ax.ticklabel_format(style='plain')
plt.show()
# plt.savefig('./visualization/거래가격 분포.png')

# 거래가격분포 logscale
fig = plt.figure(figsize=(10, 5))
sns.histplot(np.log1p(train_df['target']))
plt.title("거래가격 분포(logscale)", fontsize=14)
ax = plt.gca()
ax.ticklabel_format(style='plain')
plt.show()
# plt.savefig('./visualization/거래가격 분포(logscale).png')

# 전용면적 분포 확인
fig = plt.figure(figsize=(10, 5))
sns.histplot(train_df['전용면적(㎡)'])
plt.title("전용면적 분포", fontsize=14)
plt.show()
# plt.savefig('./visualization/전용면적 분포.png')

# 전용면적과 거래가격의 관계
fig = plt.figure(figsize=(10, 10))
sns.scatterplot(train_df, x=train_df['target'], y=train_df['전용면적(㎡)'], color='salmon')
plt.title("전용면적과 거래가격의 관계", fontsize=14)
plt.show()
# plt.savefig('./visualization/전용면적과 거래가격의 관계.png')

# 평균 단가(㎡당 가격) 확인
# 거래가격 ÷ 전용면적
fig = plt.figure(figsize=(10, 5))
sns.histplot(train_df['target'] / train_df['전용면적(㎡)'])
plt.title("평균 단가(㎡당 가격) 분포", fontsize=14)
plt.show()
# plt.savefig('./visualization/평균 단가(㎡당 가격) 분포.png')

# 구별 평균거래가 확인
df_sorted = train_df.groupby('구')['target'].agg(가격='mean').reset_index()
df_sorted = df_sorted.sort_values('가격', ascending=False)

fig = plt.figure(figsize=(8,10))
sns.barplot(
    data=df_sorted,
    x="가격", 
    y="구", 
    palette="flare",
    legend=False
)
plt.title("서울시 구별 평균 거래가격", fontsize=14)
plt.xlabel("가격")
plt.ylabel("구")
plt.show()
# plt.savefig('./visualization/서울시 구별 평균 거래가격.png')

# 층수 분포 확인
fig = plt.figure(figsize=(10, 5))
sns.histplot(train_df['층'], binwidth=1)
plt.title("층 수 분포", fontsize=14)
plt.show()
# plt.savefig('./visualization/층 수 분포.png')

# 층수별 거래가격 확인
fig = plt.figure(figsize=(10, 5))
fp_df = train_df.groupby('층')['target'].agg(가격='mean').reset_index()
sns.barplot(fp_df, x='층', y='가격', color='Green')
plt.xticks(rotation=90)
plt.title("층 수별 거래가격", fontsize=14)
plt.ylabel('평균가격')
plt.show()
# plt.savefig('./visualization/층 수별 거래가격.png')

# 건축년도 분포
plt.figure(figsize=(10,5))
sns.histplot(train_df['건축년도'], binwidth=1)
plt.title("건축년도 분포", fontsize=14)
plt.show()
# plt.savefig('./visualization/건축년도 분포.png')

# 주택연식 분포
plt.figure(figsize=(10,5))
sns.histplot(train_df['연식'], binwidth=1)
plt.title("주택연식 분포", fontsize=14)
plt.show()
# plt.savefig('./visualization/주택연식 분포.png')

# 건축년도와 거래가격의 관계
cyear_df = train_df.groupby('건축년도')['target'].agg(가격='mean').reset_index()
plt.figure(figsize=(10,5))
sns.barplot(cyear_df, x='건축년도', y='가격', color='Orange')
plt.title("건축년도와 거래가격의 관계", fontsize=14)
plt.ylabel('평균가격')
plt.xticks(rotation=75)
plt.show()
# plt.savefig('./visualization/건축년도와 거래가격의 관계.png')

# 주택연식과 거래가격의 관계
year_df = train_df.groupby('연식')['target'].agg(가격='mean').reset_index()

plt.figure(figsize=(10,5))
sns.barplot(year_df, x='연식', y='가격', color='Orange')
plt.title("주택연식과 거래가격의 관계", fontsize=14)
plt.ylabel('평균가격')
plt.gca().invert_xaxis()
plt.xticks(rotation=75)
plt.show()
# plt.savefig('./visualization/주택연식과 거래가격의 관계.png')

# 월별 평균 거래가격
month_df = train_df.groupby('계약월')['target'].agg(가격='mean').reset_index()

plt.figure(figsize=(10,5))
sns.barplot(month_df, x='계약월', y='가격', color='Grey')
plt.title("월별 평균거래가격", fontsize=14)
plt.ylabel('평균가격')
plt.show()
# plt.savefig('./visualization/월별 평균거래가격.png')

# 아파트명 브랜드 처리
brand_patterns = {
    "래미안": r"래미안|레미안",
    "힐스테이트": r"힐스테이트|디에이치",
    "자이": r"자이",
    "e편한세상": r"e-편한세상|이편한세상|e편한세상",
    "롯데캐슬": r"롯데캐슬|르엘",
    "아이파크": r"PARK",
    "푸르지오": r"푸르지오|써밋",
    "더샵": r"더샵",
    "센트레빌": r"센트레빌",
    "스위첸": r"스위첸",
    "유보라": r"유보라",
    "주공": r"주공|엘에이치|휴먼시아"
}

# 초기화
apt['브랜드명'] = "일반"

# 브랜드별로 매칭
for brand, pattern in brand_patterns.items():
    mask = apt['아파트명'].str.contains(pattern, flags=re.IGNORECASE, regex=True)
    apt.loc[mask, '브랜드명'] = brand

# 브랜드별 평균거래가격
apt_brand = apt.groupby('브랜드명')['target'].agg(가격='mean').reset_index()
apt_sorted = apt_brand.sort_values('가격', ascending=False)

fig = plt.figure(figsize=(8,10))
sns.barplot(
    data=apt_sorted,
    x="가격", 
    y="브랜드명", 
    palette="crest",
    legend=False
)
plt.title("아파트 브랜드명별 평균 거래가격", fontsize=14)
plt.xlabel("가격")
plt.ylabel("브랜드명")
plt.show()
# plt.savefig('./visualization/아파트 브랜드명별 평균 거래가격.png')

# 강남/비강남
gangnam = train_df[train_df['강남권여부'] == 1][['연식', '강남권여부', 'target']]
n_gangnam = train_df[train_df['강남권여부'] == 0][['연식', '강남권여부', 'target']]

# 강남권/비강남권 연식별 거래가격
gangnam_df = gangnam.groupby('연식')['target'].agg(가격='mean').reset_index()
n_gangnam_df = n_gangnam.groupby('연식')['target'].agg(가격='mean').reset_index()

fig = plt.figure(figsize=(10, 5))
ax = sns.barplot(gangnam_df, x='연식', y='가격', color='skyblue', alpha=0.6)
plt.xticks(rotation=90)
# ax2 = ax.twinx()
ax = sns.barplot(n_gangnam_df, x='연식', y='가격', color='red', alpha=0.6)
plt.ylabel('평균가격')
plt.xticks(rotation=90)
plt.title("강남권/비강남권 연식별 거래가격", fontsize=14)
plt.show()
# plt.savefig('./visualization/강남권_비강남권 연식별 거래가격.png')

# 데이터의 상관계수 행렬을 생성
corr = train_df.corr(numeric_only=True, method='spearman')
mask = np.ones_like(corr, dtype=bool)
mask = np.triu(mask)

# 히트맵 형태로 상관행렬 시각화하기
plt.figure(figsize=(13,10))
sns.heatmap(data=corr, annot=True, fmt='.2f', mask=mask, linewidths=.5, cmap='RdYlBu_r') 
plt.title('Correlation Matrix')
plt.show()
# plt.savefig('./visualization/Correlation_Matrix.png')